In [ ]:
# default_exp create_database_tables

# Create Database Tables

> Creates and populates the database from a raw 'latest-all.json.bz2' file. Only run when you want to create the database again.

In [ ]:
#export
# Package to read the the JSON dump
from qwikidata.json_dump import WikidataJsonDump
from kirby.properties import properties

In [ ]:
#export
wjd = WikidataJsonDump('latest-all.json.bz2') # put the name of the JsonDump here
items_list = [] # This will hold the data for the items
items = {}  # Connected items go in the objects dictionary
max_entities = 100  # Variable to stop the program during testing
property_list = properties()  # List of all properties
aliases = []  # List of all aliases

In [ ]:
#export
# Make sure all string with apostrophes have an scape character for sql
def fix_label(label):
    index = 0
    while index < len(label):
        if label[index] == '\'':
            label = label[:index] + '\'' + label[index:]
            index += 1
        index += 1
    return label

In [ ]:
#export
f = open("sql_statements.txt", "a", encoding="utf-8")

In [ ]:
#export
for ii, entity_dict in enumerate(wjd):
    if ii % 500 == 0 and ii != 0:
        count = 0
        for item in items_list:
            if count % 1000 == 0:
                f.write("\n\n\n\nINSERT INTO Entities\n (entity_id, label, description)\n VALUES\n")
            count += 1
            if count == 1000 or item == items_list[-1]:
                item = item[:-1] + ';'
            f.write('{}\n'.format(item))
        items_list.clear()
        count = 0

        for item in property_list:
            if count % 1000 == 0:
                f.write(
                    "\n\n\n\nINSERT INTO Properties_relations\n (entity_id, property_id, related_entity_id)\n VALUES\n")
            count += 1
            if count % 1000 == 0 or item == property_list[-1]:
                item = item[:-1] + ';'
            f.write('{}\n'.format(item))
        property_list.clear()
        count == 0
        for idx, item in enumerate(aliases):
            if count % 1000 == 0:
                f.write("\n\n\n\nINSERT INTO Aliases\n (entity_id, alias)\n VALUES\n")
            count += 1
            if count % 1000 == 0 or count == len(aliases):
                item = item[:-1] + ';'
            f.write('{}\n'.format(item))

        aliases.clear()

    entity_label = ''
    description = ''
    entity_properties = []
    # Store the id of the entity
    entity_id = entity_dict['id']
    index = 0

    if 'descriptions' in entity_dict and 'en' in entity_dict['descriptions'] and 'value' in entity_dict['descriptions']['en']:
        description = entity_dict['descriptions']['en']['value']

    # Just keep the entities that have a name in English
    if 'labels' in entity_dict:
        if 'en' in entity_dict['labels']:
            # temp = fix_label(entity_dict['labels']['en']['value'])
            items_list.append('(\'{}\', \'{}\', \'{}\'),'.format(entity_id, fix_label(entity_dict['labels']['en']['value']),
                                                                 fix_label(description)))
        else:
            continue

    if 'aliases' in entity_dict:  # Create table
        if 'en-gb' in entity_dict['aliases']:
            for element in entity_dict['aliases']['en-gb']:
                aliases.append('(\'{}\', \'{}\'),'.format(entity_id, fix_label(element['value'])))
        if 'en' in entity_dict['aliases']:
            for element in entity_dict['aliases']['en']:
                aliases.append('(\'{}\', \'{}\'),'.format(entity_id, fix_label(element['value'])))

    if "claims" in entity_dict:
        for inner_key in entity_dict["claims"].keys():
            if 'datavalue' in entity_dict["claims"][inner_key][0]['mainsnak'] \
                    and ('value' in entity_dict["claims"][inner_key][0]['mainsnak']['datavalue']) \
                    and isinstance(entity_dict["claims"][inner_key][0]['mainsnak']['datavalue']['value'], dict) \
                    and ('id' in entity_dict["claims"][inner_key][0]['mainsnak']['datavalue']['value']):
                property_list.append('(\'{}\', \'{}\', \'{}\'),'.format(entity_id, inner_key, entity_dict["claims"][inner_key][0]['mainsnak']['datavalue']['value']['id']))

count = 0
for k in property_dict:
    if count % 1000 == 0:
        f.write("\n\n\n\nINSERT INTO Properties\n (property_id, label)\n VALUES\n")
    count += 1
    if count % 1000 == 0:
        f.write('(\'{}\', \'{}\');\n'.format(k, fix_label(property_dict[k])))
    else:
        f.write('(\'{}\', \'{}\'),\n'.format(k, fix_label(property_dict[k])))

f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'latest-all.json.bz2'